# GoTo ML
This is a solution for qualifying problem for GoTo ML school

6 different approaches are implemented below:
+ dummy: random prediction 
+ mk0: baseline solution from the first
+ mk1: statistical solution based on finding similarity between user and film in films.json feature space
+ mk2: SVD matrix decomposition
+ mk3: Random forest tree classification on top-k features
+ mk4: Neural net regression on top-k features

## Parts:
1. Data precalc
2. Defining first-teir solutions
2. More data precalc
3. Defining second-teir solutions
4. Quality metrics

## Functions
There are two common types of function:
+ user_film_NAME function - by user and item returns their similariy
+ recommend_NAME function - by user returns top-k recommendations for user 

In [ ]:
import csv
from collections import defaultdict, Counter
import random
import json
from math import sqrt
from metrics import apk
random.seed(42)

In [ ]:
from sklearn.ensemble import *
import numpy as np

In [ ]:
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

In [ ]:
import lasagne
import theano
import theano.tensor as T

In [ ]:
import tensorflow as tf

## Data precalc

In [ ]:
#Train
user_to_items = defaultdict(set)
item_to_users = defaultdict(set)
edges = []
#Test
test_user_to_items = defaultdict(set)
test_item_to_users = defaultdict(set)
test_edges = []

with open("data/train_likes.csv") as datafile:
    for like in csv.DictReader(datafile):
        # Every like goes to train or test
        if random.random() < 0.5:
            user_to_items[like['user_id']].add(like['item_id'])
            item_to_users[like['item_id']].add(like['user_id'])
            edges.append((like['user_id'], like['item_id']))
        else:
            test_user_to_items[like['user_id']].add(like['item_id'])
            test_item_to_users[like['item_id']].add(like['user_id'])
            test_edges.append((like['user_id'], like['item_id']))

In [ ]:
all_items = set(item_to_users.keys()) | set(test_item_to_users.keys())
all_users = set(user_to_items.keys()) | set(test_user_to_items.keys())

In [ ]:
users_list = list(user_to_items.keys())
test_users_list = list(test_user_to_items.keys())

In [ ]:
user_to_i = {user: i for i, user in enumerate(all_users)}
item_to_i = {item: i for i, item in enumerate(all_items)}
all_users_list = list(all_users)
all_items_list = list(all_items)

In [ ]:
matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
test_matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
for user, items in user_to_items.items():
    for item in items:
        matrix[user_to_i[user], item_to_i[item]] = True
for user, items in test_user_to_items.items():
    for item in items:
        test_matrix[user_to_i[user], item_to_i[item]] = True
matrix = matrix.tocsr()
test_matrix = test_matrix.tocsr()

In [ ]:
matrix

In [ ]:
u, s, vt = sp.sparse.linalg.svds(matrix.astype(np.float32), k=100)

In [ ]:
u.shape, s.shape, vt.shape

In [ ]:
films = json.load(open('data/items.json'))
films = {a['id']:a for a in films}
for a in films.values():
    del a['id']

In [ ]:
for f in films.values():
    if 'genre' in f:
        f[f['genre']] = 1
        del f['genre']

In [ ]:
#Finding out what features are more popular
features = Counter()
for f in films.values():
    features.update(set(f.keys()))
features.most_common(10)

In [ ]:
small_features_size = 60 

In [ ]:
#Map feature to it's number
small_features = set([_[0] for _ in features.most_common(small_features_size)])
feature_to_i = {feature: i for i, feature in enumerate(small_features)}

In [ ]:
#User filtration
min_items_per_user = 2
from copy import copy
for user in copy(test_user_to_items).keys():
    
    n_items_per_user = len(user_to_items[user]) + len(test_user_to_items[user])
    
    if n_items_per_user <= min_items_per_user:
        del user_to_items[user]
        del test_user_to_items[user]

## Statistical solutions

In [ ]:
def user_film_mk2(user, item, debug=False):
    ui = user_to_i[user]
    ii = item_to_i[item]
    if debug:
        print(ui, ii)
    a = np.dot(u[ui,:] * s, vt[:,ii])
    return a

In [ ]:
def generic_recommend(user_flim_function):
    'Funstion return recomend_NAME function by user_flim_NAME function'
    def recommend(user, n_best = 10, debug=False):
        user_items = user_to_items[user]

        item_similarities = {}
        for item in all_items:
            if item in user_items: continue
            item_users = item_to_users[item]
            if len(item_users) == 0: continue

            item_similarities[item] = user_flim_function(user, item)

        items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
        if debug:
            for a in  items_sorted[:n_best]:
                print((a, item_similarities.get(a, 0)))
        return items_sorted[:n_best]
    return recommend

In [ ]:
def recommend_dummy(user, n_best = 10):
    item_similarities = []
    for item in all_items:
        #пропустим те фильмы, которые пользователь уже просмотрел, если нас об этом попросили
        if item in user_to_items[user]: continue
        item_similarities.append(item)
         
    random.shuffle(item_similarities)
    #вернём n_best наиболее пригодных
    #print(items_sorted[:n_best])
    return item_similarities[:n_best]

In [ ]:
def recommend_mk0(user, n_best = 10):
    user_items = user_to_items[user]
    
    neighborhood = Counter()
    for item in user_items:
        neighborhood.update(item_to_users[item])
    
    #словарь {фильм -> пригодность фильма пользователю}
    item_similarities = {}
    
    for item in all_items:
        if item in user_items: continue
        item_users = item_to_users[item]
        if len(item_users) == 0: continue
        
        n_common_users = sum(neighborhood[user] for user in item_users)
        similarity = float(n_common_users) / sqrt(len(item_users))
        item_similarities[item] = similarity
    
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    
    return items_sorted[:n_best]

## More data precalc

In [ ]:
user_to_int = dict()
for i, user in enumerate(all_users):
    user_to_int[user] = i 

In [ ]:
#User_features is a dict of user's features from the same space as for films
user_features = defaultdict(lambda:defaultdict(lambda:0))
for user in list(user_to_items.keys())[0:]:
    for item in user_to_items[user]:
        if item in films:
            for feature, value in films[item].items():    
                user_features[user][feature]+=value
    for f in user_features[user]:
        user_features[user][f]/=len(user_to_items)

In [ ]:
features_size = 2*small_features_size + 1

In [ ]:
def extract_features(user=None, item=None, X_sample = np.zeros(features_size, dtype='float32')):
    'Returns 1 sample for specified user and film'
    if user is not None:
        for f in user_features[user].keys()&small_features:
            X_sample[feature_to_i[f]] = user_features[user][f]
        if item is not None:
            X_sample[2*small_features_size+0] = user_film_mk2(user, item)
    if item is not None and item in films:
        for f in films[item].keys()&small_features:
            X_sample[small_features_size+feature_to_i[f]] = films[item][f]
    return X_sample

In [ ]:
def user_film_mk1(user, item, debug=False):
    #print(len(user_features[user]))
    cursum = 0
    curcnt = 0
    if item in films:
        for feature, value in films[item].items():
            if type(value) is int:
                curcnt+=1    
                cursum+=value*user_features[user][feature]
    if curcnt==0:
        curcnt+=1
    #cursum/=curcnt
    if debug:
        print(cursum)
    return cursum

In [ ]:
def generate_random_samples(X_size = 100, use_test=False):
    'Generate samples randomly from any part of dataset'
    X = np.zeros((X_size, features_size), dtype='float32')
    Y = np.zeros(X_size, dtype='int8')
    if use_test:
        local_users_list = test_users_list
        local_user_to_items = test_user_to_items
    else:
        local_users_list = users_list
        local_user_to_items = user_to_items
    for i in range(X_size):
        film =''
        while film not in films: 
            result = random.random()
            if result > 0.50:
                result = 1
            else:
                result = 0
            user = random.choice(local_users_list)
            if result==0:
                film = random.choice(all_items_list)
                if film in local_user_to_items[user]:
                    result = 1
            else:
                film = random.choice(list(local_user_to_items[user]|{''}))
        X[i] = extract_features(user, film)
        Y[i] = result
    return X, Y

In [ ]:
def generate_samples(X_true = 100,  use_test=False):
    'Generate samples from begining one-by-one'
    X_fake=X_true
    X = np.zeros((X_true+X_fake, features_size), dtype='float32')
    Y = np.zeros(X_true+X_fake, dtype='float32')
    if use_test:
        local_users_list = test_users_list
        local_user_to_items = test_user_to_items
        local_edges = test_edges 
    else: 
        local_users_list = users_list
        local_user_to_items = user_to_items
        local_edges = edges
    for i, (user, film)  in enumerate(local_edges[:X_true]):
        X[i] = extract_features(user, film)
        Y[i] = 1
    for i in range(X_true, X_true+X_fake):
        user = random.choice(local_users_list)
        film = random.choice(all_items_list)
        X[i] = extract_features(user, film)
        if film in local_user_to_items[user]:
            Y[i] = 1
    return X, Y

## Random Forest solution

In [ ]:
rf = RandomForestRegressor(n_estimators=500)

In [ ]:
X, Y = generate_random_samples(100000)

In [ ]:
rf.fit(X, Y)

In [ ]:
def user_film_mk3(user, item, debug=False):
    X = np.zeros((1, features_size), dtype='float32')
    X[0] = extract_features(user, item)
    probs = rf.predict(X)
    if debug:
        print(probs)
    return probs[0]

In [ ]:
def recommend_mk3_manual(user, n_best = 10, debug=False):
    user_items = user_to_items[user]

    item_similarities = {}
    cur_items = [item for item in all_items if item not in user_items and len(item_to_users[item])>0]
    
    X = np.zeros((len(cur_items), features_size), dtype='float32')
    X_user = extract_features(user)
    for i, item in enumerate(cur_items):
        X[i] = extract_features(user=None, item=item, X_sample=X_user)
    
    probs =rf.predict(X)
    item_similarities = {key: prob for key, prob in zip(cur_items, probs)}
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    if debug:
        print(X)
        for a in  items_sorted[:n_best]:
            print((a, item_similarities.get(a, 0)))
    return items_sorted[:n_best]

In [ ]:
X, Y = generate_random_samples(100, use_test=False)
print("Train score: %s" %(rf.score(X, Y),))
X, Y = generate_random_samples(100, use_test=True)
print("Test score: %s" %(rf.score(X, Y),))

In [ ]:
X, Y = generate_random_samples(10, use_test=True)
X[0], rf.predict(X), Y

## Lasagne solution

In [ ]:
input_X = T.matrix("input X")
input_shape = [None, features_size]

target_y = T.matrix("target Y integer",dtype='float32')

In [ ]:
layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

layer = lasagne.layers.BatchNormLayer(layer)

layer = lasagne.layers.DenseLayer(layer,num_units=features_size,
                                   nonlinearity = lasagne.nonlinearities.linear,
                                   name = "hidden_dense_layer")
layer = lasagne.layers.DenseLayer(layer,num_units=features_size,
                                   nonlinearity = lasagne.nonlinearities.linear,
                                   name = "hidden_dense_layer")
layer = lasagne.layers.DenseLayer(layer,num_units=features_size,
                                   nonlinearity = lasagne.nonlinearities.sigmoid,
                                   name = "hidden_dense_layer")

layer = lasagne.layers.DropoutLayer(layer, p=0.5)

layer = lasagne.layers.DenseLayer(layer,num_units=features_size,
                                   nonlinearity = lasagne.nonlinearities.sigmoid,
                                   name = "hidden_dense_layer")
dense_output = lasagne.layers.DenseLayer(layer,num_units = 1 ,
                                        nonlinearity = lasagne.nonlinearities.sigmoid,
                                        name='output', b=None)

In [ ]:
y_predicted = lasagne.layers.get_output(dense_output)
all_weights = lasagne.layers.get_all_params(dense_output, trainable=True)

loss = lasagne.objectives.squared_error(y_predicted, target_y)
loss = lasagne.objectives.aggregate(loss, mode = 'mean') #use mean squared_error


updates = lasagne.updates.adadelta(loss, all_weights,learning_rate=0.01)
train_fun = theano.function([input_X,target_y],loss,updates= updates)

In [ ]:
y_predicted_clear = lasagne.layers.get_output(dense_output, deterministic=True)
predict = theano.function([input_X], y_predicted_clear)


loss_clear = lasagne.objectives.squared_error(y_predicted_clear, target_y)
loss_clear = lasagne.objectives.aggregate(loss_clear, mode = 'mean') #use mean squared_error
loss_fun = theano.function([input_X,target_y],loss_clear)

In [ ]:
import time

num_epochs = 3 #amount of passes through the data

batch_size = 1000 #number of samples processed at each function call
num_batches = 100

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in (generate_samples(batch_size, use_test=False) for _ in range(num_batches)):
        inputs, targets = batch
        targets = targets.reshape((-1, 1))
        train_err_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_batches = 0
    for batch in (generate_samples(batch_size, use_test=True) for _ in range(num_batches)):
        inputs, targets = batch
        targets = targets.reshape((-1, 1))
        val_err += loss_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  train loss: %s" % (train_err / train_batches, ))
    print("  test loss: %s" % (val_err / val_batches, ))
    

#### Best
test loss: 0.15436154014

In [ ]:
X, Y = generate_random_samples(10, use_test=True)
predict(X), Y

In [ ]:
def recommend_mk4_manual(user, n_best = 10, debug=False):
    user_items = user_to_items[user]

    item_similarities = {}
    cur_items = [item for item in all_items if item not in user_items and len(item_to_users[item])>0]
    
    X = np.zeros((len(cur_items), features_size), dtype='float32')
    X_user = extract_features(user)
    for i, item in enumerate(cur_items):
        X[i] = extract_features(user=None, item=item, X_sample=X_user)
    
    probs = predict(X)
    item_similarities = {key: prob for key, prob in zip(cur_items, probs)}
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    if debug:
        print(X)
        for a in  items_sorted[:n_best]:
            print((a, item_similarities.get(a, 0)))
    return items_sorted[:n_best]

## TensorFlow solution
This solution is not finished (model is working, but recommend function is absent)

In [ ]:
learning_rate = 0.01
training_epochs = 100
batch_size = 1000
total_batch = 200
display_step = 1

In [ ]:
x = tf.placeholder(tf.float32, [None, features_size]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 1]) 

W = tf.Variable(tf.zeros([features_size, 1]))
b = tf.Variable(tf.zeros([1]))

pred = tf.nn.tanh(tf.matmul(x, W) + b)

cost = tf.reduce_mean(tf.squared_difference(y, pred))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
sess = tf.Session()
sess.run(init)

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0.

    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = generate_random_samples(batch_size)
        batch_ys = batch_ys.reshape((-1, 1))
        # Fit training using batch data
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                      y: batch_ys})
        # Compute average loss
        avg_cost += c / total_batch
    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print ("Epoch:", '%04d' % (epoch+1), "cost=", "%s" % (avg_cost))
        x_test, y_test =   generate_random_samples(batch_size, use_test=True)
        y_test = y_test.reshape((-1, 1))
        print ("Train cost:", cost.eval({x: x_test, y: y_test}, session=sess))

print ("Optimization Finished!")

# Test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
x_test, y_test =   generate_random_samples(batch_size, use_test=True)
y_test = y_test.reshape((-1, 1))
print ("Train cost:", cost.eval({x: x_test, y: y_test}, session=sess))

In [ ]:
X, Y = generate_random_samples(10, use_test=False)
sess.run([pred], feed_dict={x: X}), Y

## Manual tests

In [ ]:
generic_recommend(user_film_mk1)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk1('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

In [ ]:
generic_recommend(user_film_mk2)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk2('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

In [ ]:
user_film_mk3('26052b20aa96ed8d803dbfe4e9497192', '45ea2aa2143effcb6575daf0143e31b4', debug=True)

In [ ]:
recommend_mk4_manual('a18f526db904f8f2ee4c8d178bfc818c', debug=True)

In [ ]:
list(test_user_to_items.items())[7]

# Quality check - errors

In [ ]:
#Random forest score (less - better)
X, Y = generate_random_samples(100, use_test=False)
print("Train score: %s" %(rf.score(X, Y),))
X, Y = generate_random_samples(100, use_test=True)
print("Test score: %s" %(rf.score(X, Y),))

In [ ]:
#Lasagne error
inputs, targets = generate_random_samples(100, use_test=False)
targets = targets.reshape((-1, 1))
print("Train score: %s" % (loss_fun(inputs, targets),)) 
inputs, targets = generate_samples(100, use_test=True)
targets = targets.reshape((-1, 1))
print("Test score: %s" % (loss_fun(inputs, targets),))

# Quality check - map@k

In [ ]:
#сколько рекоммендаций рассматриваем
def check_quality(function, K = 10, max_n_users = len(test_user_to_items)):
    APatK_per_user = []
    user_list = list(test_user_to_items.keys())[:max_n_users]
    
    for i, user in enumerate(user_list):
        #фильмы, которые пользователю на самом деле нравятся
        test_items = test_user_to_items[user]

        #Выдать топ-K рекоммендаций
        recommendation_list = function(user,n_best=K)
        #Посчитать ap@k
        user_APatK = apk(test_items, recommendation_list,k=K)

        #и сложить в коробку
        APatK_per_user.append(user_APatK)

        #Progress bar
        if i % 100 ==0:
            print(i,'/',max_n_users)

        if i > max_n_users:
            break

    print('AP@{} = {}'.format(K, sum(APatK_per_user)/len(APatK_per_user)))

In [ ]:
check_quality(recommend_dummy,100,200)

In [ ]:
check_quality(recommend_mk0,100, 200)

In [ ]:
check_quality(generic_recommend(user_film_mk1),100, 200)

In [ ]:
check_quality(generic_recommend(user_film_mk2),200,200)

In [ ]:
check_quality(recommend_mk3_manual,100, 200)

In [ ]:
check_quality(recommend_mk4_manual,100, 200)